<a href="https://colab.research.google.com/github/Narenderreddyranabothu/DataScience-Project/blob/main/P_and_O_of_smart_grids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# This will open a file picker dialog
uploaded = files.upload()

Saving household_power_consumption.txt to household_power_consumption.txt


In [2]:
import warnings
warnings.filterwarnings("ignore") # For suppressing warnings

import pandas as pd  # For data handling
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For plotting graphs
import seaborn as sns  # For prettier graphs
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.ensemble import RandomForestRegressor  # For building a prediction model
from sklearn.metrics import mean_squared_error, mean_absolute_error  # For evaluating the model
from datetime import datetime  # For handling dates
from sklearn.preprocessing import StandardScaler  # For scaling data
import statsmodels.api as sm  # For statistical analysis

In [3]:
# Load the dataset
df = pd.read_csv('household_power_consumption.txt', sep=';', low_memory=False)

In [4]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [5]:
print('Total number of data points in the dataset:', len(df))

Total number of data points in the dataset: 2075259


# Data Cleaning


In [6]:
missing_values_count = df.isnull().sum()
total_missing = missing_values_count.sum()

print("Missing values per column:\n", missing_values_count)
print(f"\nTotal missing values in dataset: {total_missing}")

Missing values per column:
 Date                         0
Time                         0
Global_active_power          0
Global_reactive_power        0
Voltage                      0
Global_intensity             0
Sub_metering_1               0
Sub_metering_2               0
Sub_metering_3           25979
dtype: int64

Total missing values in dataset: 25979


In [7]:
df = df.dropna()

In [8]:
missing_values_count = df.isnull().sum()
total_missing = missing_values_count.sum()

print("Missing values per column:\n", missing_values_count)
print(f"\nTotal missing values in dataset: {total_missing}")

Missing values per column:
 Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

Total missing values in dataset: 0


In [9]:
print('Total number of data points in the dataset (Missing values removed):', len(df))

Total number of data points in the dataset (Missing values removed): 2049280


In [10]:
dx = df.copy()

In [11]:
dx['Datetime'] = pd.to_datetime(dx['Date'] + ' ' + dx['Time'], dayfirst=True)

# Set Datetime as index
dx.set_index('Datetime', inplace=True)

# Drop the old Date and Time columns safely
dx = dx.drop(['Date', 'Time'], axis=1)

# Convert all columns to numeric, coercing errors (non-numeric to NaN)
for col in dx.columns:
    dx[col] = pd.to_numeric(dx[col], errors='coerce')

# Now resample by hour and take the mean
dx = dx.resample('h').mean()

In [12]:
dx = dx.reset_index()
dx.head()

,Datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:00:00,4.222889,0.229000,234.643889,18.100000,0.0,0.527778,16.861111
1,2006-12-16 18:00:00,3.632200,0.080033,234.580167,15.600000,0.0,6.716667,16.866667
2,2006-12-16 19:00:00,3.400233,0.085233,233.232500,14.503333,0.0,1.433333,16.683333
3,2006-12-16 20:00:00,3.268567,0.075100,234.071500,13.916667,0.0,0.000000,16.783333
4,2006-12-16 21:00:00,3.056467,0.076667,237.158667,13.046667,0.0,0.416667,17.216667


In [13]:
print('Total number of data points in the dataset (Update: Minutes to Hourly):', len(dx))

Total number of data points in the dataset (Update: Minutes to Hourly): 34589


# Feature Engineering


In [14]:
# Total Sub-Metered Energy
dx['Total_sub_metering'] = dx['Sub_metering_1'] + dx['Sub_metering_2'] + dx['Sub_metering_3']

In [15]:
dx.head()

,Datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Total_sub_metering
0,2006-12-16 17:00:00,4.222889,0.229000,234.643889,18.100000,0.0,0.527778,16.861111,17.388889
1,2006-12-16 18:00:00,3.632200,0.080033,234.580167,15.600000,0.0,6.716667,16.866667,23.583333
2,2006-12-16 19:00:00,3.400233,0.085233,233.232500,14.503333,0.0,1.433333,16.683333,18.116667
3,2006-12-16 20:00:00,3.268567,0.075100,234.071500,13.916667,0.0,0.000000,16.783333,16.783333
4,2006-12-16 21:00:00,3.056467,0.076667,237.158667,13.046667,0.0,0.416667,17.216667,17.633333


In [16]:
# Global Active Power is in kilowatts. Convert it to watt-minutes per minute interval (1 kW = 1000 watts)
dx['Global_active_energy'] = dx['Global_active_power'] * 1000 / 60  # Watt-minute per minute

# Unmetered Energy - residual power use not captured by sub-metering
dx['Unmetered_energy'] = dx['Global_active_energy'] - dx['Total_sub_metering']

In [17]:
dx.head()

,Datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Total_sub_metering,Global_active_energy,Unmetered_energy
0,2006-12-16 17:00:00,4.222889,0.229000,234.643889,18.100000,0.0,0.527778,16.861111,17.388889,70.381481,52.992593
1,2006-12-16 18:00:00,3.632200,0.080033,234.580167,15.600000,0.0,6.716667,16.866667,23.583333,60.536667,36.953333
2,2006-12-16 19:00:00,3.400233,0.085233,233.232500,14.503333,0.0,1.433333,16.683333,18.116667,56.670556,38.553889
3,2006-12-16 20:00:00,3.268567,0.075100,234.071500,13.916667,0.0,0.000000,16.783333,16.783333,54.476111,37.692778
4,2006-12-16 21:00:00,3.056467,0.076667,237.158667,13.046667,0.0,0.416667,17.216667,17.633333,50.941111,33.307778


In [18]:
# Reactive/Active Power Ratio
# Helps in analysing power quality and energy efficiency

dx['Power_factor_proxy'] = dx['Global_reactive_power'] / (dx['Global_active_power'] + 1e-6)  # adding a small noise to avoid division by zero

In [19]:
# Load Categories
# Categorising total power usage levels

dx['Load_category'] = pd.cut(dx['Global_active_power'], bins = [0, 2, 4, dx['Global_active_power'].max()],
                             labels = ['Low', 'Medium', 'High'])

In [20]:
dx.head()

,Datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Total_sub_metering,Global_active_energy,Unmetered_energy,Power_factor_proxy,Load_category
0,2006-12-16 17:00:00,4.222889,0.229000,234.643889,18.100000,0.0,0.527778,16.861111,17.388889,70.381481,52.992593,0.054228,High
1,2006-12-16 18:00:00,3.632200,0.080033,234.580167,15.600000,0.0,6.716667,16.866667,23.583333,60.536667,36.953333,0.022034,Medium
2,2006-12-16 19:00:00,3.400233,0.085233,233.232500,14.503333,0.0,1.433333,16.683333,18.116667,56.670556,38.553889,0.025067,Medium
3,2006-12-16 20:00:00,3.268567,0.075100,234.071500,13.916667,0.0,0.000000,16.783333,16.783333,54.476111,37.692778,0.022976,Medium
4,2006-12-16 21:00:00,3.056467,0.076667,237.158667,13.046667,0.0,0.416667,17.216667,17.633333,50.941111,33.307778,0.025083,Medium
